# Multilabel Text Classification using CNN

This notebook demonstrates multilabel classification using a Convolutional Neural Network (CNN).

Dataset: GoEmotions (simplified), a large dataset of Reddit comments labeled with 28 emotion categories.

The model uses:
- Embedding layer for word representations
- Conv1D layers to capture local features
- Sigmoid activation for multilabel output
- Binary cross-entropy loss

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install datasets tensorflow torch --quiet

In [2]:
from datasets import load_dataset
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

## 1. Dataset Loading & Exploration

We load the GoEmotions simplified dataset.

In [8]:
# Load dataset
dataset = load_dataset("go_emotions", "simplified", split="train")

texts = dataset['text']
raw_labels = dataset['labels']  # Multilabel: list of integers

# Get the emotion class list
class_list = dataset.features['labels'].feature.names

## 2. Text Preprocessing & Tokenization

Text is tokenized using TensorFlow/Keras Tokenizer.

In [9]:
# Convert to multi-hot
mlb = MultiLabelBinarizer(classes=class_list)
y = mlb.fit_transform([[class_list[i] for i in lbl] for lbl in raw_labels])

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
X_seq = tokenizer.texts_to_sequences(texts)
X_pad = pad_sequences(X_seq, maxlen=150, padding='post')

## 3. CNN Model Architecture

The CNN model consists of:

- Embedding layer
- Multiple Conv1D layers with ReLU activation
- Global Max Pooling
- Dense layers with sigmoid output

The model predicts probabilities for each emotion label independently.

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.metrics import AUC, Precision, Recall

model = Sequential([
    Embedding(input_dim=10000, output_dim=64),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(class_list), activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')]
)

model.build(input_shape=(None, X_pad.shape[1]))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 150, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 146, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 28)             │         1,820 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 691,164 (2.64 MB)

 Trainable params: 691,164 (2.64 MB)

 Non-trainable params: 0 (0.00 B)

## 4. Model Training & Evaluation

We train the CNN model with binary cross-entropy loss and evaluate using multilabel metrics.

In [11]:
model.fit(X_pad, y, epochs=15, batch_size=32, validation_split=0.2)

Epoch 1/15
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 35s 30ms/step - accuracy: 0.2957 - auc: 0.7345 - loss: 0.1888 - precision: 0.2442 - recall: 0.0737 - val_accuracy: 0.4656 - val_auc: 0.8826 - val_loss: 0.1155 - val_precision: 0.6531 - val_recall: 0.3480
Epoch 2/15
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 41s 30ms/step - accuracy: 0.4767 - auc: 0.8890 - loss: 0.1130 - precision: 0.7033 - recall: 0.3199 - val_accuracy: 0.5039 - val_auc: 0.9084 - val_loss: 0.1058 - val_precision: 0.6816 - val_recall: 0.3740
Epoch 3/15
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 40s 37ms/step - accuracy: 0.5283 - auc: 0.9253 - loss: 0.0985 - precision: 0.7377 - recall: 0.3845 - val_accuracy: 0.5189 - val_auc: 0.9129 - val_loss: 0.1040 - val_precision: 0.6751 - val_recall: 0.3990
Epoch 4/15
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 31s 29ms/step - accuracy: 0.5710 - auc: 0.9427 - loss: 0.0882 - precision: 0.7682 - recall: 0.4315 - val_accuracy: 0.5168 - val_auc: 0.9137 - val_loss: 0.1037 - val_precision: 0.6874 - val_recall: 0.3774
Epoch 5/15
1086/

## 5. Inference with sample 

In [12]:
sample_plot = ["A young boy discovers he has magical powers and attends a school for wizards."]
sample_seq = tokenizer.texts_to_sequences(sample_plot)
sample_pad = pad_sequences(sample_seq, maxlen=150, padding='post')

pred_probs = model.predict(sample_pad)
pred_labels = (pred_probs >= 0.5).astype(int)
predicted_genres = mlb.inverse_transform(pred_labels)
print("Predicted genres:", predicted_genres)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Predicted genres: [('neutral',)]


## 6. Conclusion & Next Steps

CNNs provide a powerful alternative to traditional ML for multilabel text classification.

Next steps could be:

- Adding pre-trained embeddings (e.g., GloVe)
- Using attention mechanisms
- Experimenting with more complex architectures (e.g., LSTM, transformers)